In [1]:
import matplotlib
import scipy 
import sys
import argparse
import numpy
import dadi
from dadi import Numerics, PhiManip, Integration, Spectrum, Misc
import dadi.Godambe

In [2]:
fs = dadi.Spectrum.from_file("ENP-GOC.sfs")
# check if it's folded, if not folded, fold it
if fs.folded==False:
    fs=fs.fold()
else:
    fs=fs
ns = fs.sample_sizes # get sample size from SFS (in haploids)
pts_l = [ns[0]+5,ns[0]+15,ns[0]+25] # need 6 points because two populations
maxiter=100

In [3]:
#Bootstraping
datafile = './Bootstrap/SNPs_for_SFS.vcf.gz'
dd = dadi.Misc.make_data_dict_vcf(datafile, './Bootstrap/Samples.txt')

In [4]:
# Generate 100 bootstrap datasets, by dividing the genome into 2 Mb chunks and
# resampling from those chunks.
Nboot, chunk_size = 100, 2e6
chunks = dadi.Misc.fragment_data_dict(dd, chunk_size)

In [5]:
pop_ids, ns = ['ENP','GOC'], [44,30]
boots = dadi.Misc.bootstraps_from_dd_chunks(chunks, Nboot, pop_ids, ns, polarized=False)

## Uncertainty analysis 2D.Split.SimMig

In [11]:
def split_WAmig(params, ns, pts):
    nu1, nu2, m, TDiv = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = PhiManip.phi_1D_to_2D(xx, phi)  # split into two pops

    phi = Integration.two_pops(phi, xx, TDiv, nu1, nu2, m12=m, m21=m)  # two pops at diff sizes with asymmetric mig
    fs = Spectrum.from_phi(phi, ns, (xx,xx))
    return fs

func=split_WAmig
func_ex = dadi.Numerics.make_extrap_log_func(func)
# nu1, nu2, m, TDiv
popt = [1.22312738,0.054469858,6.700956776, 0.036973726] 

uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [2.36198330e-02 1.58958694e-03 1.34402814e-01 1.95217540e-03
 1.39037351e+04]


## Uncertainty analysis 2D.Split.NoMig

In [10]:
def split_WAmig(params, ns, pts):
    nu1, nu2, TDiv = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = PhiManip.phi_1D_to_2D(xx, phi)  # split into two pops

    phi = Integration.two_pops(phi, xx, TDiv, nu1, nu2, m12=0, m21=0)  # two pops at diff sizes with asymmetric mig
    fs = Spectrum.from_phi(phi, ns, (xx,xx))
    return fs

func=split_WAmig
func_ex = dadi.Numerics.make_extrap_log_func(func)
# nu1, nu2, TDiv
popt = [9.719370868,0.018649682,0.007322175] 

uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [1.74897271e+01 4.93955545e-03 1.91624769e-03 1.38956143e+04]


## Uncertainty analysis 2D.Split.ConstantSize.AsyMig

In [16]:
def split_WAmig(params, ns, pts):
    nu1, nu2, m12, m21, TDiv = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = PhiManip.phi_1D_to_2D(xx, phi)  # split into two pops

    phi = Integration.two_pops(phi, xx, TDiv, nu1, nu2, m12=m12, m21=m21)  # two pops at diff sizes with asymmetric mig
    fs = Spectrum.from_phi(phi, ns, (xx,xx))
    return fs

func=split_WAmig
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [17]:
# nu1, nu2, m12, m21, TDiv
popt = [1.312681056,0.007127091,2.31597455,109.4102365, 0.22589947]

In [18]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [1.87815985e-02 5.71920206e-04 1.58007786e-01 9.37064151e+00
 2.91063051e-02 1.50781302e+04]


## Uncertainty analysis 2D.Split.ConstantSize.AsyMig.Tcon2gen

In [6]:
def split_WAmig(params, ns, pts):
    nu1a,nu1b, nu2, m12, m21, TDiv = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = PhiManip.phi_1D_to_2D(xx, phi)  # split into two pops

    phi = Integration.two_pops(phi, xx, TDiv, nu1a, nu2, m12=m12, m21=m21)
    phi = Integration.two_pops(phi, xx, 0.000065, nu1b, nu2, m12=m12, m21=m21)
    fs = Spectrum.from_phi(phi, ns, (xx,xx))
    return fs

func=split_WAmig
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [7]:
# nu1a,nu1b, nu2, m12, m21, TDiv
popt = [1.467576568,0.024608236,0.003446829,4.29113141,227.62587,0.16501407] 
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [8.63468865e-03 1.02078260e-03 9.62650563e-05 1.49386613e-01
 6.39359892e+00 3.32050179e-03 7.91444974e+02]


In [8]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [2.93591986e-02 3.04924432e-03 1.22756428e-04 4.16422340e-01
 9.63472974e+00 2.23725176e-02 1.47310971e+04]


## Uncertainty analysis 2D.Split.ConstantSize.AsyMig.Tcon3gen

In [12]:
def split_WAmig(params, ns, pts):
    nu1a,nu1b, nu2, m12, m21, TDiv = params
    xx = Numerics.default_grid(pts) # sets up grid
    phi = PhiManip.phi_1D(xx) # sets up initial phi for population
    phi = PhiManip.phi_1D_to_2D(xx, phi)  # split into two pops

    phi = Integration.two_pops(phi, xx, TDiv, nu1a, nu2, m12=m12, m21=m21)
    phi = Integration.two_pops(phi, xx, 0.00011, nu1b, nu2, m12=m12, m21=m21)
    fs = Spectrum.from_phi(phi, ns, (xx,xx))
    return fs

func=split_WAmig
func_ex = dadi.Numerics.make_extrap_log_func(func)
# nu1a,nu1b, nu2, m12, m21, TDiv
popt = [1.4593447,0.042139905,0.003747356,4.080661268,209.2673019,0.166033176] 
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [2.72872735e-02 4.88263077e-03 1.50997582e-04 3.75076668e-01
 9.75769487e+00 2.23689560e-02 1.46692580e+04]


## Uncertainty analysis 2D.Anc.Split.AsyMig

In [19]:
def asym_mig_size(params, ns, pts):


    Ta, nua, Td, nu1, nu2, m12, m21 = params

    xx = Numerics.default_grid(pts)

    phi = PhiManip.phi_1D(xx)

    phi = Integration.one_pop(phi, xx, T=Ta, nu=nua) #Ancetral size change

    phi = PhiManip.phi_1D_to_2D(xx, phi)

    phi = Integration.two_pops(phi, xx, Td, nu1=nu1, nu2=nu2,m12=m12, m21=m21) #Divergence time


    fs = Spectrum.from_phi(phi, ns, (xx,xx))

    return fs

func=asym_mig_size
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [20]:
# Ta, nua, Tdv, nu1, nu2, m12, m21
popt = [0.113302768,1.542119587,0.018799237,1.063072805,0.006960227, 3.020005007, 111.6928484]

In [21]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [3.63613833e-02 1.46688166e-01 6.93613029e-03 9.87365831e-02
 4.42775861e-04 3.98663478e-01 7.53033256e+00 1.50508669e+04]


In [12]:
def asym_mig_size(params, ns, pts):


    Ta, Td, nu1, nu2, m12, m21 = params

    xx = Numerics.default_grid(pts)

    phi = PhiManip.phi_1D(xx)

    phi = Integration.one_pop(phi, xx, T=Ta, nu=1.5) #Ancetral size change

    phi = PhiManip.phi_1D_to_2D(xx, phi)

    phi = Integration.two_pops(phi, xx, Td, nu1=nu1, nu2=nu2,m12=m12, m21=m21) #Divergence time


    fs = Spectrum.from_phi(phi, ns, (xx,xx))

    return fs

func=asym_mig_size
func_ex = dadi.Numerics.make_extrap_log_func(func)
popt = [0.111980071,0.019266442,1.064750758,0.007051611,2.998634174,110.2652561] 
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [8.11512085e-03 3.15219646e-03 5.13094848e-02 5.90577115e-04
 3.06799902e-01 9.46752818e+00 1.38730065e+04]


## Uncertainty analysis 2D.Anc.Split.AsyMig.Isolation

In [22]:
def asym_mig_size(params, ns, pts):


    Ta, nua, Td, nu1, nu2,Tc, m12, m21 = params

    xx = Numerics.default_grid(pts)

    phi = PhiManip.phi_1D(xx)

    phi = Integration.one_pop(phi, xx, T=Ta, nu=nua) #Ancetral size change

    phi = PhiManip.phi_1D_to_2D(xx, phi)

    phi = Integration.two_pops(phi, xx, Td, nu1=nu1, nu2=nu2,m12=0, m21=0)

    phi = Integration.two_pops(phi, xx, Tc, nu1=nu1, nu2=nu2,m12=m12, m21=m21)
    
    fs = Spectrum.from_phi(phi, ns, (xx,xx))

    return fs

func=asym_mig_size
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [23]:
# Ta, nua, Tdv, nu1, nu2, Tc, m12, m21
popt = [0.175926396,1.396884816,0.003266979,0.844074075,0.006017274,0.00134638,9.91875635,147.0897349] 


In [24]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [1.77182864e-01 1.94026700e-01 2.36847628e-04 4.88900099e-01
 3.07924203e-04 2.13294025e-04 7.47082266e+00 4.15273857e+00
 2.92590355e+04]


In [16]:
def asym_mig_size(params, ns, pts):


    Td, nu1, Tc, m12, m21 = params

    xx = Numerics.default_grid(pts)

    phi = PhiManip.phi_1D(xx)

    phi = Integration.one_pop(phi, xx, T=0.1133, nu=1.542) #Ancetral size change

    phi = PhiManip.phi_1D_to_2D(xx, phi)

    phi = Integration.two_pops(phi, xx, Td, nu1=nu1, nu2=0.0069,m12=0, m21=0)

    phi = Integration.two_pops(phi, xx, Tc, nu1=nu1, nu2=0.0069,m12=m12, m21=m21)
    
    fs = Spectrum.from_phi(phi, ns, (xx,xx))

    return fs

func=asym_mig_size
func_ex = dadi.Numerics.make_extrap_log_func(func)
popt = [0.003818878,0.677868223,0.001385141,14.35202077,133.0047679] 
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [1.23627813e-04 4.23027716e-02 2.49643457e-04 2.42284441e+00
 6.75007456e+00 1.32277758e+04]


## Uncertainty analysis 2D.Anc.Split.AsyMig.Pacific

In [17]:
def asym_mig_size(params, ns, pts):


    Ta, nua, Td, nu1a, nu1b, nu2,Tc, m12, m21 = params

    xx = Numerics.default_grid(pts)

    phi = PhiManip.phi_1D(xx)

    phi = Integration.one_pop(phi, xx, T=Ta, nu=nua) #Ancetral size change

    phi = PhiManip.phi_1D_to_2D(xx, phi)

    phi = Integration.two_pops(phi, xx, Td, nu1=nu1a, nu2=nu2,m12=m12, m21=m21)

    phi = Integration.two_pops(phi, xx, Tc, nu1=nu1b, nu2=nu2,m12=m12, m21=m21)
    
    fs = Spectrum.from_phi(phi, ns, (xx,xx))

    return fs

func=asym_mig_size
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [18]:
# Ta, nua, Tdv, nu1a,nu1b, nu2, Tc, m12, m21
popt = [1.865792047,1.292489112,0.068710184,2.566908166,1.124041955,0.007855384,0.022014248,2.609872484,99.18682611] 
uncerts_fim = dadi.Godambe.FIM_uncert(func_ex, pts_l, popt, fs, multinom=True)
print('Estimated parameter standard deviations from FIM: {0}'.format(uncerts_fim))

Estimated parameter standard deviations from FIM: [3.10225635e-01 6.47001738e-02            nan            nan
 5.25347273e-02 2.89963836e-04 6.44473398e-04 1.19888693e-01
 3.65958137e+00 2.86108493e+04]


In [ ]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

## Uncertainty analysis 2D.Anc.Split.AsyMig.Gulf

In [13]:
def asym_mig_size(params, ns, pts):


    Ta, nua, Td, nu1, nu2a, nu2b,Tc, m12, m21 = params

    xx = Numerics.default_grid(pts)

    phi = PhiManip.phi_1D(xx)

    phi = Integration.one_pop(phi, xx, T=Ta, nu=nua) #Ancetral size change

    phi = PhiManip.phi_1D_to_2D(xx, phi)

    phi = Integration.two_pops(phi, xx, Td, nu1=nu1, nu2=nu2a,m12=m12, m21=m21)

    phi = Integration.two_pops(phi, xx, Tc, nu1=nu1, nu2=nu2b,m12=m12, m21=m21)
    
    fs = Spectrum.from_phi(phi, ns, (xx,xx))

    return fs

func=asym_mig_size
func_ex = dadi.Numerics.make_extrap_log_func(func)

In [14]:
# Ta, nua, Tdv, nu1,nu2a, nu2b, Tc, m12, m21
popt = [0.14286067,1.447490772,0.00279061,0.691394501,0.003682354,0.008799041,0.001023487,5.678478249,109.5230839] 

In [15]:
uncerts = dadi.Godambe.GIM_uncert(func_ex, pts_l, boots, popt, fs, 
                                  multinom=True)
print('Estimated parameter standard deviations from GIM: {0}'.format(uncerts))

Estimated parameter standard deviations from GIM: [6.30048542e-02 1.37024929e-01 2.19145862e-03 4.04086603e-01
 2.26337762e-03 4.01680101e-03 5.87907686e-04 2.92821779e+00
 4.92778594e+01 1.72803815e+04]
